In [1]:
from flask import Flask, jsonify, request
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD


In [4]:
app = Flask(__name__)

# Load the Amazon product review dataset
amazon_ratings = pd.read_csv('ratings_Beauty.csv')
amazon_ratings = amazon_ratings.dropna()

# Function to build the recommendation system
def build_recommendation_system():
    # Popularity-based recommendation system
    popular_products = pd.DataFrame(amazon_ratings.groupby('ProductId')['Rating'].count())
    most_popular = popular_products.sort_values('Rating', ascending=False)

    # Model-based collaborative filtering system
    amazon_ratings1 = amazon_ratings.head(10000)
    ratings_utility_matrix = amazon_ratings1.pivot_table(values='Rating', index='UserId', columns='ProductId', fill_value=0)
    X = ratings_utility_matrix.T

    # Singular Value Decomposition (SVD)
    SVD = TruncatedSVD(n_components=10)
    decomposed_matrix = SVD.fit_transform(X)

    # Correlation Matrix
    correlation_matrix = np.corrcoef(decomposed_matrix)

    return most_popular, X, correlation_matrix

# Function to get recommendations for a given product ID
def get_recommendations(product_id, most_popular, X, correlation_matrix):
    # Popularity-based recommendations
    popularity_recommendations = most_popular.head(10).index.tolist()

    # Model-based collaborative filtering recommendations
    product_names = list(X.index)
    product_ID = product_names.index(product_id)
    correlation_product_ID = correlation_matrix[product_ID]
    model_based_recommendations = list(X.index[correlation_product_ID > 0.90])
    model_based_recommendations.remove(product_id)

    return popularity_recommendations, model_based_recommendations

# Route to get recommendations for a given product ID
@app.route('/recommend/<string:product_id>', methods=['GET'])
def recommend(product_id):
    most_popular, X, correlation_matrix = build_recommendation_system()
    popularity_recommendations, model_based_recommendations = get_recommendations(product_id, most_popular, X, correlation_matrix)

    return jsonify({
        "popularity_recommendations": popularity_recommendations,
        "model_based_recommendations": model_based_recommendations
    })

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
